# Trace and Call Trees

## 1. Setup Workspace

In [1]:
import json
import pathlib
import copy
import textwrap

In [2]:
data_dir = pathlib.Path.cwd().parent / "data"
trace_file = data_dir / "fibonacci" / "paptrace.json"
with open(trace_file, "r") as f_in:
    traces = json.load(f_in)["traces"]

In [3]:
class Node:
    
    def __init__(self, id_, data):
        self.id = id_
        self.data = data
        self.parent = None
        self.children = []
        
    def add_child(self, child):
        child.parent = self
        self.children.append(child)
        
    def depth(self):
        depth = 0
        curr_parent = self.parent
        while curr_parent is not None:
            depth += 1
            curr_parent = curr_parent.parent
        return depth
        
        
    def __str__(self):
        depth = self.depth()
        leader = "" if depth == 0 else f"{' ' * 4 * (depth - 1)}└── "
        s = f"{leader}{self.id}: {self.data}"
        for c in self.children:
            s += f"\n{c.__str__()}"
        return s

## 2. Captured Traces as Trees

In this section we convert the traces, as captured, to trees.

In [4]:
def get_trace_desc(trace):
    if "sig" in trace:
        sig = trace["sig"]
        params = ", ".join([f"{p['name']}={p['value']}" for p in trace["params"]])
        return f"{sig} @ ({params})"
    return trace["desc"]


def trace_to_tree(trace):
    node = Node(trace["id"], get_trace_desc(trace))
    if "children" in trace:
        for child in trace["children"]:
            node.add_child(trace_to_tree(child))
    return node


trees = []
for trace in traces:
    trees.append(trace_to_tree(trace))
    
print(trees[0], "\n")
print(trees[1], "\n")
print(trees[2], "\n")

2106190: unsigned long long fibonacci::RecursiveNaive(unsigned short) @ (n=0)
└── 2106009: n < 2
    └── 2106007: return n 

2106190: unsigned long long fibonacci::RecursiveNaive(unsigned short) @ (n=1)
└── 2106009: n < 2
    └── 2106007: return n 

2106190: unsigned long long fibonacci::RecursiveNaive(unsigned short) @ (n=2)
└── 2106188: return RecursiveNaive(n - 1) + RecursiveNaive(n - 2)
    └── 2106190: unsigned long long fibonacci::RecursiveNaive(unsigned short) @ (n=1)
        └── 2106009: n < 2
            └── 2106007: return n
    └── 2106190: unsigned long long fibonacci::RecursiveNaive(unsigned short) @ (n=0)
        └── 2106009: n < 2
            └── 2106007: return n 



# 3. Extract Calls

In this section we walk the traces to extract call nodes.

In [5]:
def extract_calls(calls, trace):
    if "children" in trace:
        for child in trace["children"]:
            extract_calls(calls, child)
    if "Call" in trace["type"]:
        call_node = copy.copy(trace)
        child_nodes = {"children": call_node["children"]}
        del call_node["children"]
        calls.append((call_node, child_nodes))
        
calls = []
for trace in traces:
    extract_calls(calls, trace)
    
print("CALL: ", calls[2][0], "\nPATH: ", calls[2][1], "\n")
print("CALL: ", calls[3][0], "\nPATH: ", calls[3][1], "\n")
print("CALL: ", calls[4][0], "\nPATH: ", calls[4][1], "\n")

CALL:  {'id': 2106190, 'params': [{'name': 'n', 'value': '1'}], 'sig': 'unsigned long long fibonacci::RecursiveNaive(unsigned short)', 'type': 'CalleeExpr'} 
PATH:  {'children': [{'children': [{'children': [], 'desc': 'return n', 'id': 2106007, 'type': 'ReturnStmt'}], 'desc': 'n < 2', 'id': 2106009, 'type': 'IfThenStmt'}]} 

CALL:  {'id': 2106190, 'params': [{'name': 'n', 'value': '0'}], 'sig': 'unsigned long long fibonacci::RecursiveNaive(unsigned short)', 'type': 'CalleeExpr'} 
PATH:  {'children': [{'children': [{'children': [], 'desc': 'return n', 'id': 2106007, 'type': 'ReturnStmt'}], 'desc': 'n < 2', 'id': 2106009, 'type': 'IfThenStmt'}]} 

CALL:  {'id': 2106190, 'params': [{'name': 'n', 'value': '2'}], 'sig': 'unsigned long long fibonacci::RecursiveNaive(unsigned short)', 'type': 'CalleeExpr'} 
PATH:  {'children': [{'children': [{'children': [{'children': [{'children': [], 'desc': 'return n', 'id': 2106007, 'type': 'ReturnStmt'}], 'desc': 'n < 2', 'id': 2106009, 'type': 'IfThenSt

In [6]:
print(f"Number of call nodes (original): {len(calls)}")

# Remove all non-unique call nodes.
uniq_calls = []
for call in calls:
    if call not in uniq_calls:
        uniq_calls.append(call)
calls = uniq_calls

print(f"Number of call nodes (unique): {len(calls)}")

Number of call nodes (original): 288
Number of call nodes (unique): 184


In [7]:
# Arrange calls by signature.
call_map = {}
for call, path in calls:
    ctx_map = call_map.setdefault(call["sig"], {})
    param_list = tuple([p["value"] for p in call["params"]])
    path_list = ctx_map.setdefault(param_list, [])
    if path not in path_list:
        path_list.append(path)
    
def print_call_map(call_map):
    for sig, ctx_map in call_map.items():
        print(f"sig: {sig}")
        for ctx, paths in ctx_map.items():
            print(f"└── params: {', '.join(ctx)}")
            if len(paths) > 1:
                raise Exception("Detected more than one unique path for params {ctx}.")
            print(f"{' ' * 4}└── path: {paths[0]}")
        print()

    
print_call_map(call_map)

sig: unsigned long long fibonacci::RecursiveNaive(unsigned short)
└── params: 0
    └── path: {'children': [{'children': [{'children': [], 'desc': 'return n', 'id': 2106007, 'type': 'ReturnStmt'}], 'desc': 'n < 2', 'id': 2106009, 'type': 'IfThenStmt'}]}
└── params: 1
    └── path: {'children': [{'children': [{'children': [], 'desc': 'return n', 'id': 2106007, 'type': 'ReturnStmt'}], 'desc': 'n < 2', 'id': 2106009, 'type': 'IfThenStmt'}]}
└── params: 2
    └── path: {'children': [{'children': [{'children': [{'children': [{'children': [], 'desc': 'return n', 'id': 2106007, 'type': 'ReturnStmt'}], 'desc': 'n < 2', 'id': 2106009, 'type': 'IfThenStmt'}], 'id': 2106190, 'params': [{'name': 'n', 'value': '1'}], 'sig': 'unsigned long long fibonacci::RecursiveNaive(unsigned short)', 'type': 'CalleeExpr'}, {'children': [{'children': [{'children': [], 'desc': 'return n', 'id': 2106007, 'type': 'ReturnStmt'}], 'desc': 'n < 2', 'id': 2106009, 'type': 'IfThenStmt'}], 'id': 2106190, 'params': [{'name

In [8]:
# Display the call map using trees.
for sig, ctx_map in call_map.items():
    print(f"sig: {sig}")
    for ctx, paths in ctx_map.items():
        print(f"└── params: {', '.join(ctx)}")
        path = paths[0] # There should only be one path per ctx.
        print(f"{' ' * 4}└── path:")
        path_str = "\n".join([f"{trace_to_tree(c)}" for c in path["children"]])
        print(textwrap.indent(path_str, ' '*8))
    print()

sig: unsigned long long fibonacci::RecursiveNaive(unsigned short)
└── params: 0
    └── path:
        2106009: n < 2
        └── 2106007: return n
└── params: 1
    └── path:
        2106009: n < 2
        └── 2106007: return n
└── params: 2
    └── path:
        2106188: return RecursiveNaive(n - 1) + RecursiveNaive(n - 2)
        └── 2106190: unsigned long long fibonacci::RecursiveNaive(unsigned short) @ (n=1)
            └── 2106009: n < 2
                └── 2106007: return n
        └── 2106190: unsigned long long fibonacci::RecursiveNaive(unsigned short) @ (n=0)
            └── 2106009: n < 2
                └── 2106007: return n
└── params: 3
    └── path:
        2106188: return RecursiveNaive(n - 1) + RecursiveNaive(n - 2)
        └── 2106190: unsigned long long fibonacci::RecursiveNaive(unsigned short) @ (n=2)
            └── 2106188: return RecursiveNaive(n - 1) + RecursiveNaive(n - 2)
                └── 2106190: unsigned long long fibonacci::RecursiveNaive(unsigned short) 